# ResNet Training on 15% random sample of the entire data set

In [0]:
# model and learning
import numpy as np
import h5py
import cv2 as cv


%tensorflow_version 2.x
import tensorflow as tf
import keras as ks

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.optimizers import SGD
from keras.callbacks import CSVLogger


import os as os
import pathlib as pl
from PIL import ImageFile

from google.colab import drive

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.2.0


Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')
my_data_dir = '/content/drive/My Drive/DataForColab/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## TPU Set Up

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Initializing the TPU system: grpc://10.53.68.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.68.210:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Model

In [0]:
def main(withGPU=True):
    LOCATION = my_data_dir + 'Artworks'
    PATH = pl.Path(LOCATION)
    IMAGES = PATH
    CLASSES = [item.name for item in PATH.glob('*')]
    CLASSES.sort()
    BATCH_SIZE = 256
    EPOCHS = 20
    IMG_WIDTH = 200
    IMG_HEIGHT = 200
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    image_generator = ks.preprocessing.image.ImageDataGenerator(rescale=1. / 255,
                                                                featurewise_center=True,
                                                                featurewise_std_normalization=True)

    image_generator.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape((1, 1, 3))  # ordering: [R, G, B]
    image_generator.std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape((1, 1, 3))

    train_data_gen = image_generator.flow_from_directory(directory=IMAGES,
                                                         batch_size=BATCH_SIZE,
                                                         shuffle=True,
                                                         target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                         color_mode='rgb',
                                                         class_mode='categorical',
                                                         classes=CLASSES,
                                                         interpolation='lanczos')
    
    X_val, y_val = next(train_data_gen)

    input_tensor = ks.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    base_model = InceptionResNetV2(input_tensor=input_tensor, weights='imagenet', include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = ks.layers.GlobalAveragePooling2D(data_format="channels_last")(x)
    x = ks.layers.BatchNormalization()(x)
    # TODO: ADD FLATTEN
    # let's add a fully-connected layer
    x = ks.layers.Dense(256, activation='relu', kernel_regularizer=ks.regularizers.l2(l=0.001))(x)
    x = ks.layers.Dropout(rate=0.5)(x)
    x = ks.layers.Dense(256, activation='relu', kernel_regularizer=ks.regularizers.l2(l=0.001))(x)
    # and a logistic layer
    predictions = ks.layers.Dense(len(CLASSES), activation='softmax')(x)

    csv_logger = CSVLogger(my_data_dir + "model_randomsamp_history_log.csv", append=True)

    if withGPU:
      with strategy.scope():
        model = ks.Model(inputs=base_model.input, outputs=predictions)
        
        for layer in base_model.layers:
          layer.trainable = False
          
        model.compile(loss='categorical_crossentropy',
                      optimizer=ks.optimizers.Adam(lr=2e-03),
                      metrics=['accuracy', 'top_k_categorical_accuracy'])
    else:
      model = ks.models.Model(inputs=base_model.input, outputs=predictions)
      for layer in base_model.layers:
        layer.trainable = False
      model.compile(loss='categorical_crossentropy',
                    optimizer=ks.optimizers.Adam(lr=2e-03),
                    metrics=['accuracy', 'top_k_categorical_accuracy'])

    model.fit_generator(train_data_gen, epochs=EPOCHS, validation_data=(X_val, y_val),
                      callbacks=[csv_logger], verbose=1)

    model.save(my_data_dir + "save_model")

    return model, train_data_gen

In [0]:
model_output, data_gen = main(True)

Found 24694 images belonging to 3058 classes.
219062272/219055592 [==============================] - 2s 0us/step
Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 46 could not be retrieved. It could be because a worker has died.
  UserWarning)


 1/97 [..............................] - ETA: 2:04:02 - loss: 8.7973 - accuracy: 0.0000e+00 - top_k_categorical_accuracy: 0.0000e+00

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 31 could not be retrieved. It could be because a worker has died.
  UserWarning)


 2/97 [..............................] - ETA: 1:51:49 - loss: 8.7487 - accuracy: 0.0020 - top_k_categorical_accuracy: 0.0020        

...

95/97 [============================>.] - ETA: 2:29 - loss: 7.8123 - accuracy: 0.0282 - top_k_categorical_accuracy: 0.0790

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 8 could not be retrieved. It could be because a worker has died.
  UserWarning)


96/97 [============================>.] - ETA: 1:14 - loss: 7.8065 - accuracy: 0.0287 - top_k_categorical_accuracy: 0.0798

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 52 could not be retrieved. It could be because a worker has died.
  UserWarning)


97/97 [==============================] - 7302s 75s/step - loss: 7.8009 - accuracy: 0.0287 - top_k_categorical_accuracy: 0.0799 - val_loss: 7.3982 - val_accuracy: 0.0312 - val_top_k_categorical_accuracy: 0.0703
Epoch 2/20
97/97 [==============================] - 360s 4s/step - loss: 6.9054 - accuracy: 0.0558 - top_k_categorical_accuracy: 0.1351 - val_loss: 6.9251 - val_accuracy: 0.0234 - val_top_k_categorical_accuracy: 0.1016
Epoch 3/20
97/97 [==============================] - 359s 4s/step - loss: 6.4051 - accuracy: 0.0701 - top_k_categorical_accuracy: 0.1640 - val_loss: 6.6143 - val_accuracy: 0.0430 - val_top_k_categorical_accuracy: 0.1250
Epoch 4/20
97/97 [==============================] - 359s 4s/step - loss: 6.0845 - accuracy: 0.0835 - top_k_categorical_accuracy: 0.1919 - val_loss: 6.4264 - val_accuracy: 0.0742 - val_top_k_categorical_accuracy: 0.1836
Epoch 5/20
97/97 [==============================] - 359s 4s/step - loss: 5.8169 - accuracy: 0.0948 - top_k_categorical_accuracy: 0.21